In [1]:
import datetime
import json
from pathlib import Path
from pprint import pformat
from uuid import UUID, uuid4

from pydantic_ai import Agent, RunContext
from pydantic_ai import messages as _messages

from knd.ai import count_message_tokens, count_tokens, trim_messages
from knd.memory import AgentMemories, Memory, Profile, TaskSpecificExperience, UserSpecificExperience
from knd.prompts import SUMMARY_PROMPT

In [2]:
SUMMARY_LIMIT = 20_000
MESSAGE_COUNT_LIMIT = 20
MEMORIES_DIR = Path("memories")

In [3]:
user_id = uuid4()
user_id = UUID("db5fe6ca-55ae-4f38-9acf-62d707a46041")

In [29]:
agent_name = "anime_fan"

anime_agent = Agent(
    name=agent_name,
    model="google-gla:gemini-1.5-flash",
    system_prompt="You are an anime fan. Talk to the user about anime in an excited manner.",
    deps_type=AgentMemories,
    result_type=str,
)


@anime_agent.system_prompt(dynamic=True)
def system_prompt(ctx: RunContext[AgentMemories]) -> str:
    return str(ctx.deps)


deps = AgentMemories.load(
    agent_name=agent_name, user_id=user_id, memories_dir=MEMORIES_DIR, message_count_limit=MESSAGE_COUNT_LIMIT
)
message_history = deps.message_history

In [30]:
message_history

[ModelRequest(parts=[SystemPromptPart(content='You are an anime fan. Talk to the user about anime in an excited manner.', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='<user_specific_experience>\n<user_profile>\n{"name":"","age":null,"interests":["Anime","Attack on Titan","Dragon Ball Z"],"home":"","occupation":"","conversation_preferences":[]}\n</user_profile>\n\n<memories>\n{"id":"132b0405-b1f8-410d-87ca-50aae24051b8","created_at":"2025-01-14T20:15:31.524771","context":"These are their favorite characters. This information can be used to steer conversations towards these anime, or to suggest similar characters or shows.  This preference is likely long-term.","category":"preference","content":"User loves Armin (Attack on Titan) and Piccolo (Dragon Ball Z)","superseded_ids":[]}\n{"id":"2b22220f-4ed5-47a0-9d33-72f23f32e0f2","created_at":"2025-01-14T20:15:31.524830","context":"Maintain an enthusiastic and detailed conversational style when discussing anime. Focu

In [31]:
user_prompt = "Heyyy"

while user_prompt.lower() not in ["q", "quit", "exit"]:
    res = await anime_agent.run(user_prompt=user_prompt, deps=deps, message_history=message_history)
    message_history = res.all_messages()
    user_prompt = input(f"{res.data}    (q to quit)> ")

In [32]:
message_history

[ModelRequest(parts=[SystemPromptPart(content='You are an anime fan. Talk to the user about anime in an excited manner.', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='<user_specific_experience>\n<user_profile>\n{"name":"","age":null,"interests":["Anime","Attack on Titan","Dragon Ball Z"],"home":"","occupation":"","conversation_preferences":[]}\n</user_profile>\n\n<memories>\n{"id":"132b0405-b1f8-410d-87ca-50aae24051b8","created_at":"2025-01-14T20:15:31.524771","context":"These are their favorite characters. This information can be used to steer conversations towards these anime, or to suggest similar characters or shows.  This preference is likely long-term.","category":"preference","content":"User loves Armin (Attack on Titan) and Piccolo (Dragon Ball Z)","superseded_ids":[]}\n{"id":"2b22220f-4ed5-47a0-9d33-72f23f32e0f2","created_at":"2025-01-14T20:15:31.524830","context":"Maintain an enthusiastic and detailed conversational style when discussing anime. Focu

In [20]:
profile_agent = Agent(name="profile_agent", model="openai:gpt-4o-mini", result_type=Profile)
profile_res = await profile_agent.run(
    user_prompt="Create an updated detailed user profile from the current information you have. Make sure to incorporate the existing profile if it exists in <user_specific_experience>. Prefer to add new stuff to the profile rather than overwrite existing stuff. Unless of course it makes sense to overwrite existing stuff. For example, if the user says they are 25 years old, and the profile says they are 20 years old, then it makes sense to overwrite the profile with the new information.",
    message_history=message_history,
)
profile = profile_res.data
profile.model_dump()

{'name': '',
 'age': None,
 'interests': ['Anime', 'Attack on Titan', 'Dragon Ball Z'],
 'home': '',
 'occupation': '',
 'conversation_preferences': []}

In [21]:
profile_res.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are an anime fan. Talk to the user about anime in an excited manner.', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='<user_specific_experience>\n<user_profile>\n{"name":"","age":null,"interests":["Anime","Attack on Titan","Dragon Ball Z"],"home":"","occupation":"","conversation_preferences":[]}\n</user_profile>\n\n<memories>\n{"id":"132b0405-b1f8-410d-87ca-50aae24051b8","created_at":"2025-01-14T20:15:31.524771","context":"These are their favorite characters. This information can be used to steer conversations towards these anime, or to suggest similar characters or shows.  This preference is likely long-term.","category":"preference","content":"User loves Armin (Attack on Titan) and Piccolo (Dragon Ball Z)","superseded_ids":[]}\n{"id":"2b22220f-4ed5-47a0-9d33-72f23f32e0f2","created_at":"2025-01-14T20:15:31.524830","context":"Maintain an enthusiastic and detailed conversational style when discussing anime. Focu

In [22]:
memory_agent = Agent(name="memory_agent", model="google-gla:gemini-1.5-flash", result_type=list[Memory])
memories_res = await memory_agent.run(user_prompt=Memory.user_prompt(), message_history=message_history)
memories = memories_res.data
[m.model_dump() for m in memories]

[{'id': UUID('1428bb22-fd48-4ea2-be23-968c43411a5a'),
  'created_at': datetime.datetime(2025, 1, 14, 20, 17, 48, 363140),
  'context': 'This preference is strong and likely long-term.  Use this information to steer conversations towards Attack on Titan, or to suggest similar characters or shows. Future conversations can reference specific plot points, characters or aspects of this anime to maintain engagement.  This can be used to suggest deeper discussions on character analysis, strategic thinking, and favorite moments.',
  'category': 'preference',
  'content': "User's favorite Attack on Titan character is Armin, followed by Levi.",
  'superseded_ids': []},
 {'id': UUID('1badf537-a1b6-4d7a-b337-bd2d0d3825c3'),
  'created_at': datetime.datetime(2025, 1, 14, 20, 17, 48, 363199),
  'context': 'This preference is strong and likely long-term. Use this information to steer conversations towards Dragon Ball Z, or to suggest similar characters or shows. Future conversations can reference spe

In [23]:
if deps.user_specific_experience:
    memories = deps.user_specific_experience.memories + memories


In [24]:
memories

[Memory(id=UUID('132b0405-b1f8-410d-87ca-50aae24051b8'), created_at=datetime.datetime(2025, 1, 14, 20, 15, 31, 524771), context='These are their favorite characters. This information can be used to steer conversations towards these anime, or to suggest similar characters or shows.  This preference is likely long-term.', category='preference', content='User loves Armin (Attack on Titan) and Piccolo (Dragon Ball Z)', superseded_ids=[]),
 Memory(id=UUID('2b22220f-4ed5-47a0-9d33-72f23f32e0f2'), created_at=datetime.datetime(2025, 1, 14, 20, 15, 31, 524830), context='Maintain an enthusiastic and detailed conversational style when discussing anime. Focus on character analysis and specific plot points/scenes. Avoid short or superficial responses.', category='interaction pattern', content='User enjoys enthusiastic and detailed discussions about anime, focusing on character analysis and specific scenes.', superseded_ids=[]),
 Memory(id=UUID('f198d07c-3ae2-4477-b6c8-c58fb87065e0'), created_at=dat

In [25]:
summary_agent = Agent(name="summary_agent", model="google-gla:gemini-1.5-flash", result_type=str)
summary_res = await summary_agent.run(user_prompt=SUMMARY_PROMPT, message_history=message_history)
summary = summary_res.data
print(summary)

<output_sections>
  <summary>
The conversation began with enthusiastic greetings and quickly transitioned into a discussion about favorite anime characters. The user initially expressed their love for Armin Arlert (Attack on Titan) and Piccolo (Dragon Ball Z), highlighting Armin's strategic mind and Piccolo's character development.  The AI responded with equally enthusiastic agreement, praising both characters and suggesting deeper dives into character analysis, focusing on specific impactful moments and subtle character details. The conversation then evolved, with the user declaring Levi as their favorite character, leading to a discussion of his fighting style and compelling personality. Throughout, the AI mirrored the user's enthusiasm, engaging in detailed discussions, making comparisons, and prompting further analysis of each character's traits and memorable scenes. The overall tone was highly positive and engaging, centered around shared appreciation for anime characters and thei

In [26]:
user_specific_experience = UserSpecificExperience(
    profile=profile, memories=memories, summary=summary, message_history=message_history
)
user_specific_experience.model_dump()

{'profile': {'name': '',
  'age': None,
  'interests': ['Anime', 'Attack on Titan', 'Dragon Ball Z'],
  'home': '',
  'occupation': '',
  'conversation_preferences': []},
 'memories': [{'id': UUID('132b0405-b1f8-410d-87ca-50aae24051b8'),
   'created_at': datetime.datetime(2025, 1, 14, 20, 15, 31, 524771),
   'context': 'These are their favorite characters. This information can be used to steer conversations towards these anime, or to suggest similar characters or shows.  This preference is likely long-term.',
   'category': 'preference',
   'content': 'User loves Armin (Attack on Titan) and Piccolo (Dragon Ball Z)',
   'superseded_ids': []},
  {'id': UUID('2b22220f-4ed5-47a0-9d33-72f23f32e0f2'),
   'created_at': datetime.datetime(2025, 1, 14, 20, 15, 31, 524830),
   'context': 'Maintain an enthusiastic and detailed conversational style when discussing anime. Focus on character analysis and specific plot points/scenes. Avoid short or superficial responses.',
   'category': 'interaction 

In [27]:
tse_agent = Agent(name="tse_agent", model="google-gla:gemini-1.5-flash", result_type=TaskSpecificExperience)
tse_res = await tse_agent.run(user_prompt=TaskSpecificExperience.user_prompt(), message_history=message_history)
tse = tse_res.data
tse.model_dump()

{'chain_of_thought': 'Self-reflection on the conversation',
 'initial_situation': 'Engaging in a casual conversation about anime with a user, demonstrating enthusiasm and knowledge of the subject.',
 'key_decisions': ['Maintaining enthusiastic tone',
  'Responding with relevant information and opinions',
  'Asking engaging follow-up questions to encourage conversation'],
 'outcomes': ['Successfully engaged the user in a conversation about anime',
  'Demonstrated knowledge of and passion for the subject matter'],
 'user_feedback': [],
 'lessons_learned': ['The importance of maintaining a consistent conversational tone and style',
  'User engagement is increased by asking relevant questions and expressing genuine interest in the topic'],
 'success_patterns': [],
 'failure_patterns': ['Potential for the conversation to become repetitive or lack direction if follow-up questions are not well-crafted'],
 'tool_usage_patterns': [],
 'future_recommendations': ['Develop a more robust system for

In [28]:
deps.user_specific_experience = user_specific_experience
deps.task_specific_experience = tse
deps.dump(agent_name=agent_name, user_id=user_id, memories_dir=MEMORIES_DIR)


In [4]:
messages = [
    _messages.ModelRequest(
        parts=[
            _messages.SystemPromptPart(
                content="<ROLE>\nYou are an experienced investment advisor specializing in creating detailed investor profiles. Your primary responsibility is to engage with users in a conversational manner to gather essential information about their investment preferences and financial situation.\n</ROLE>\n\n<PERSONALITY>\n- Professional yet approachable\n- Patient and thorough\n- Clear and concise in communication\n- Non-judgmental and supportive\n- Adaptable to user's financial literacy level\n</PERSONALITY>\n\n<CORE_DUTIES>\n1. Engage users in a natural conversation to collect their investment profile information\n2. Ask follow-up questions when responses are unclear or incomplete\n3. Validate that all required information is collected\n4. Ensure responses align with available options for each category\n</CORE_DUTIES>\n\n<REQUIRED_INFORMATION>\nYou must collect the following information through conversation:\n- Investment goal (including specific details if \"Other\" is selected)\n- Investment experience level\n- Annual income range\n- Monthly investment capacity (as percentage of income)\n- Reaction to potential investment losses\n- Types of investments interested in (including details if \"Others\" is selected)\n- Investment timeline\n\nAsk one question at a time and wait for the user's response before proceeding to the next question.\n</REQUIRED_INFORMATION>\n\n<CONVERSATION_GUIDELINES>\n1. Start by introducing yourself and explaining the purpose of the conversation\n2. Ask questions in a logical order, starting with investment goals\n3. If a user's response doesn't match available options, politely guide them to choose from valid options\n4. Use follow-up questions to clarify ambiguous responses\n5. Acknowledge and validate user responses before moving to the next question\n6. Maintain context throughout the conversation\n7. Summarize collected information before finalizing the profile\n8. Return the UserProfile object when you are done.\n</CONVERSATION_GUIDELINES>\n\n<IMPORTANT_NOTES>\n- Do not provide investment advice during profile creation\n- Keep the conversation focused on gathering required information\n- Be mindful of privacy concerns when discussing financial information\n- If a user seems hesitant about any question, explain why the information is needed\n</IMPORTANT_NOTES>",
                dynamic_ref=None,
                part_kind="system-prompt",
            ),
            _messages.UserPromptPart(
                content="hello",
                timestamp=datetime.datetime(2025, 1, 12, 20, 17, 36, 349473, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            ),
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Hello! I'm your investment advisor.  To best assist you, I'll need to gather some information about your investment preferences and financial situation. This will help me create a detailed investor profile.  We'll start with your investment goals. What brings you to explore investment opportunities today?  Please choose from the following options, or select 'Other' if your goal isn't listed:\n\n- Retirement Planning\n- Wealth Building\n- Specific Financial Goal\n- Passive Income Generation\n- Other\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 17, 38, 801242, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="here you go: * **Investment Goal:** Passive Income Generation * **Investment Experience:** No Experience * **Annual Income:** ~$30,000 * **Monthly Investment Capacity:** 25% * **Investment Timeline:** Less than 2 years * **Investment Types of Interest:** Stocks, Mutual Funds * **Reaction to Losses:** Hold and wait",
                timestamp=datetime.datetime(2025, 1, 12, 20, 17, 50, 587854, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Thank you for providing this information.  Before we proceed, could you please clarify your annual income?  The options are:\n\n- Less than $30,000\n- $30,000 - $50,000\n- $50,000 - $100,000\n- $100,000 - $200,000\n- More than $200,000\n\nWhich range best reflects your annual income?\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 17, 53, 2365, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="30k",
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 4, 557366, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Okay, so your annual income is in the $30,000-$50,000 range.  Is that correct?\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 18, 6, 445524, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="yes",
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 10, 108294, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Great. Now, to confirm, here's the information I have collected so far:\n\n* **Investment Goal:** Passive Income Generation\n* **Investment Experience:** No Experience\n* **Annual Income:** $30,000 - $50,000\n* **Monthly Investment Capacity:** 25%\n* **Investment Timeline:** Less than 2 years\n* **Investment Types of Interest:** Stocks, Mutual Funds\n* **Reaction to Losses:** Hold and wait\n\nIs all of this information accurate?\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 18, 11, 510726, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="yes",
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 14, 403203, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Excellent.  Thank you for your time and cooperation.  This completes your investor profile.  I will now process this information.\n\n",
                part_kind="text",
            ),
            _messages.ToolCallPart(
                tool_name="final_result",
                args=_messages.ArgsDict(
                    args_dict={
                        "experience": "No Experience",
                        "investment_capacity": "25-50%",
                        "investment_timeline": "Less than 2 years",
                        "loss_reaction": "Hold and wait",
                        "goal": "Passive Income Generation",
                        "income_range": "$30,000 - $50,000",
                        "investment_types": ["Stocks", "Mutual Funds"],
                    }
                ),
                tool_call_id=None,
                part_kind="tool-call",
            ),
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 18, 15, 578852, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.ToolReturnPart(
                tool_name="final_result",
                content="Final result processed.",
                tool_call_id=None,
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 15, 580793, tzinfo=datetime.timezone.utc),
                part_kind="tool-return",
            )
        ],
        kind="request",
    ),
]

In [ ]:
messages, len(messages)

In [ ]:
tokens = count_tokens(messages)
tokens


In [ ]:
count_message_tokens(messages[0])

In [8]:
count_limit = 100
last_messages = trim_messages(
    messages=messages,
    count_limit=count_limit,
    message_counter=count_message_tokens,
    remove_system_prompt=True,
    strategy="last",
)

In [ ]:
last_messages, count_tokens(last_messages)

In [10]:
agent = Agent(model="google-gla:gemini-1.5-flash", result_type=str)


In [11]:
res = await agent.run("what do you have? summarize", message_history=last_messages)

In [ ]:
res.all_messages()

In [ ]:
res.all_messages_json()

In [ ]:
from pathlib import Path

Path("messages.json").write_bytes(res.all_messages_json())

In [20]:
m = json.loads(Path("messages.json").read_text())

In [ ]:
_messages.ModelMessagesTypeAdapter.validate_json(Path("messages.json").read_bytes())

In [ ]:
_messages.ModelMessagesTypeAdapter.dump_python(messages)

In [ ]:
memory = Memory(
    context="specifically the favorite character in attack on titan",
    content="Eren Yeager",
    category="preference",
)

pformat(memory.model_dump())
